In [1]:
import gymnasium as gym

observations = []
questions = []
env = gym.make("room_env:RoomEnv-v2")
(obs, question), info = env.reset()
observations.append(obs)
questions.append(question)

done = False
while True:
    (obs, question), reward, done, truncated, info = env.step(("foo", "stay"))
    observations.append(obs)
    questions.append(question)
    if done:
        break


/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:491: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:156: UserWarning: WARN: The obs returned by the `res

In [2]:
from explicit_memory.nn import LSTM

kwargs_lstm = {
    "hidden_size": 64,
    "num_layers": 2,
    "n_actions": 5,
    "embedding_dim": 32,
    "capacity": {"episodic": 16, "semantic": 16, "short": 16},
    "entities": env.entities,
    "relations": env.relations,
    "include_human": "v2",
    "batch_first": True,
    "human_embedding_on_object_location": "v2",
    "device": "cpu",
}

lstm = LSTM(**kwargs_lstm)


AttributeError: 'bool' object has no attribute 'lower'

In [2]:
env.objects


{'static': [StaticObject(name: bed, location: bedroom),
  StaticObject(name: desk, location: officeroom),
  StaticObject(name: table, location: livingroom)],
 'independent': [IndependentObject(name: tae, location: livingroom, attached: []),
  IndependentObject(name: michael, location: bedroom, attached: []),
  IndependentObject(name: vincent, location: officeroom, attached: [])],
 'dependent': [DependentObject(name: laptop, location: livingroom, attached: None),
  DependentObject(name: phone, location: livingroom, attached: None),
  DependentObject(name: headset, location: bedroom, attached: None)],
 'agent': [Agent(name: agent, location: livingroom)]}

In [10]:
env.entities, env.relations


(['bed',
  'desk',
  'table',
  'tae',
  'michael',
  'vincent',
  'laptop',
  'phone',
  'headset',
  'agent'],
 ['north', 'east', 'south', 'west', 'atlocation'])

In [4]:
env.relations


['north', 'east', 'south', 'west', 'atlocation']

In [13]:
obj.name


'agent'

In [10]:
objs


[Agent(name: agent, location: livingroom)]

In [3]:
observations


[[['livingroom', 'north', 'wall', 0],
  ['livingroom', 'east', 'wall', 0],
  ['livingroom', 'south', 'bedroom', 0],
  ['livingroom', 'west', 'officeroom', 0],
  ['table', 'atlocation', 'livingroom', 0],
  ['michael', 'atlocation', 'livingroom', 0],
  ['phone', 'atlocation', 'livingroom', 0],
  ['agent', 'atlocation', 'livingroom', 0]],
 [['livingroom', 'north', 'wall', 1],
  ['livingroom', 'east', 'wall', 1],
  ['livingroom', 'south', 'bedroom', 1],
  ['livingroom', 'west', 'officeroom', 1],
  ['table', 'atlocation', 'livingroom', 1],
  ['agent', 'atlocation', 'livingroom', 1]],
 [['livingroom', 'north', 'wall', 2],
  ['livingroom', 'east', 'wall', 2],
  ['livingroom', 'south', 'bedroom', 2],
  ['livingroom', 'west', 'officeroom', 2],
  ['table', 'atlocation', 'livingroom', 2],
  ['vincent', 'atlocation', 'livingroom', 2],
  ['headset', 'atlocation', 'livingroom', 2],
  ['agent', 'atlocation', 'livingroom', 2]],
 [['livingroom', 'north', 'wall', 3],
  ['livingroom', 'east', 'wall', 3],

In [4]:
questions


[['bedroom', '?', 'wall', 0],
 ['livingroom', '?', 'officeroom', 1],
 ['livingroom', 'south', '?', 2],
 ['?', 'south', 'wall', 3],
 ['livingroom', '?', 'wall', 4],
 ['?', 'south', 'wall', 5],
 ['officeroom', 'north', '?', 6],
 ['livingroom', 'east', '?', 7],
 ['?', 'atlocation', 'bedroom', 8],
 ['officeroom', 'south', '?', 9],
 ['livingroom', 'south', '?', 10],
 ['officeroom', 'north', '?', 11],
 ['?', 'atlocation', 'livingroom', 12],
 ['?', 'west', 'wall', 13],
 ['livingroom', 'south', '?', 14],
 ['?', 'south', 'wall', 15],
 ['bedroom', '?', 'wall', 16],
 ['?', 'south', 'wall', 17],
 ['?', 'atlocation', 'bedroom', 18],
 ['livingroom', '?', 'officeroom', 19],
 ['michael', '?', 'bedroom', 20],
 ['vincent', '?', 'livingroom', 21],
 ['livingroom', 'south', '?', 22],
 ['headset', '?', 'officeroom', 23],
 ['officeroom', '?', 'livingroom', 24],
 ['desk', 'atlocation', '?', 25],
 ['desk', 'atlocation', '?', 26],
 ['?', 'west', 'officeroom', 27],
 ['vincent', '?', 'livingroom', 28],
 ['?', 'at

In [6]:
import logging

logger = logging.getLogger()
logger.disabled = True

import numpy as np
from tqdm.auto import tqdm

from agent import HandcraftedAgent


capacity = {"episodic": 16, "semantic": 16, "short": 16}

config = {
    "question_prob": 1.0,
    "seed": 42,
    "terminates_at": 99,
}

results = {}

for memory_management_policy in ["generalize", "random"]:
    for qa_policy in ["episodic_semantic", "random"]:
        for explore_policy in ["avoid_walls", "random"]:
            key = (memory_management_policy, qa_policy, explore_policy)
            if key not in results:
                results[key] = []
            print(
                memory_management_policy,
                qa_policy,
                explore_policy,
            )

            for seed in tqdm([0, 1, 2, 3, 4]):
                config["seed"] = seed

                agent = HandcraftedAgent(
                    env_str="room_env:RoomEnv-v2",
                    env_config=config,
                    memory_management_policy=memory_management_policy,
                    qa_policy=qa_policy,
                    explore_policy=explore_policy,
                    num_samples_for_results=10,
                    capacity=capacity,
                )
                agent.test()
                to_append = (np.mean(agent.scores), np.std(agent.scores))
                print(to_append)
                results[key].append(to_append)


generalize episodic_semantic avoid_walls


  0%|          | 0/5 [00:00<?, ?it/s]

/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:156: UserWarning: WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'tuple'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(68.0, 4.0)
(69.6, 8.2365041127896)
(70.2, 7.972452571197899)
(71.0, 6.148170459575759)
(67.8, 4.686149805543993)
generalize episodic_semantic random


  0%|          | 0/5 [00:00<?, ?it/s]

...

(44.8, 12.998461447417537)
(45.6, 8.662563131083086)
(45.0, 12.36931687685298)
(56.8, 8.726969691708572)
(55.2, 7.166589146867566)
random random avoid_walls


  0%|          | 0/5 [00:00<?, ?it/s]

(17.2, 6.337191807101944)
(15.0, 8.683317338436964)
(7.6, 11.551623262554921)
(10.2, 13.4)
(13.4, 12.297967311714565)
random random random


  0%|          | 0/5 [00:00<?, ?it/s]

(17.2, 6.337191807101944)
(15.0, 8.683317338436964)
(7.6, 11.551623262554921)
(10.2, 13.4)
(13.4, 12.297967311714565)


In [7]:
for key, val in results.items():
    print(
        key,
        "\t\t\t\t",
        round(np.mean([v[0] for v in val]), 2),
        round(np.std([v[0] for v in val]), 2),
    )


('generalize', 'episodic_semantic', 'avoid_walls') 				 69.32 1.24
('generalize', 'episodic_semantic', 'random') 				 60.28 2.21
('generalize', 'random', 'avoid_walls') 				 33.44 2.41
('generalize', 'random', 'random') 				 21.84 2.73
('random', 'episodic_semantic', 'avoid_walls') 				 49.48 5.35
('random', 'episodic_semantic', 'random') 				 49.48 5.35
('random', 'random', 'avoid_walls') 				 12.68 3.41
('random', 'random', 'random') 				 12.68 3.41


In [7]:
np.mean([v[0] for v in val]), np.std([v[0] for v in val])

(11.040000000000003, 5.276968826892953)